## Marker Selection

In [2]:
import pathlib
from concurrent.futures import ProcessPoolExecutor, as_completed
from itertools import combinations

import anndata
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc
import seaborn as sns
import random
from cemba_data.tools.hdf5.anndata import rank_features_groups

## Parameter

In [3]:
adata_path = '/home/hanliu/project/Linnarson_Mouse_Brain/raw/OLF.CPM.log1p.for_integration.h5ad'

cluster_col = 'SubType'

cpu = 5
top_n = 20
adj_p_cutoff = 1e-3
log2fc_cutoff = 1
min_cluster_cell_number = 10

### Stable Parameter

In [4]:
max_test_cell_population = 1000
random_seed = 0
chunk_size = 100
exclude_str = []
output_dir = 'Markers'
output_dir = pathlib.Path(output_dir)
output_dir.mkdir(exist_ok=True)

## Load Data

### Adata

In [5]:
adata = anndata.read_h5ad(adata_path)
adata

AnnData object with n_obs × n_vars = 7795 × 27998 
    obs: 'Age', 'AnalysisPool', 'AnalysisProject', 'Bucket', 'CellConc', 'Cell_Conc', 'ChipID', 'Class', 'ClassProbability_Astrocyte', 'ClassProbability_Astrocyte,Immune', 'ClassProbability_Astrocyte,Neurons', 'ClassProbability_Astrocyte,Oligos', 'ClassProbability_Astrocyte,Vascular', 'ClassProbability_Bergmann-glia', 'ClassProbability_Blood', 'ClassProbability_Blood,Vascular', 'ClassProbability_Enteric-glia', 'ClassProbability_Enteric-glia,Cycling', 'ClassProbability_Ependymal', 'ClassProbability_Ex-Neurons', 'ClassProbability_Ex-Vascular', 'ClassProbability_Immune', 'ClassProbability_Immune,Neurons', 'ClassProbability_Immune,Oligos', 'ClassProbability_Neurons', 'ClassProbability_Neurons,Cycling', 'ClassProbability_Neurons,Oligos', 'ClassProbability_Neurons,Satellite-glia', 'ClassProbability_Neurons,Vascular', 'ClassProbability_OEC', 'ClassProbability_Oligos', 'ClassProbability_Oligos,Cycling', 'ClassProbability_Oligos,Vascular', 'Cla

### Cell Tidy Data

In [6]:
cell_tidy_data = adata.obs.copy()

In [9]:
cluster_series = cell_tidy_data[cluster_col].astype(str)
cluster_counts = cluster_series.value_counts()


def check_cluster(cluster, count):
    if count < min_cluster_cell_number:
        return False
    for exclude in exclude_str:
        if exclude in cluster:
            return False
    return True


unique_clusters = [
    cluster for cluster, count in cluster_counts.items()
    if check_cluster(cluster, count)
]
cluster_pairs = list(combinations(unique_clusters, 2))
random.shuffle(cluster_pairs)  # shuffle make parallel more even

print(len(unique_clusters), 'pass filter.')
print(len(cluster_pairs), 'pairwise comparison to test.')

12 pass filter.
66 pairwise comparison to test.


## Pairwise test

In [10]:
def get_sig_features(rank_gene_dict):
    pvals_adj = pd.DataFrame(rank_gene_dict['pvals_adj'])
    names = pd.DataFrame(rank_gene_dict['names'])
    logfoldchanges = pd.DataFrame(rank_gene_dict['logfoldchanges'])
    gene_set = set(
        names.where((pvals_adj < adj_p_cutoff)
                    & (logfoldchanges.abs() > log2fc_cutoff)).values.flat)
    return gene_set


def pairwise_tests(pairs):
    adata = anndata.read_h5ad(adata_path)
    total_markers = set()
    pair_marker_count = {}
    for pair in pairs:
        cluster_a, cluster_b = pair
        cluster_a_cells = cluster_series[cluster_series == cluster_a]
        if cluster_a_cells.size > max_test_cell_population:
            cluster_a_cells = cluster_a_cells.sample(max_test_cell_population,
                                                     random_state=random_seed)
        cluster_a_cells = cluster_a_cells.index

        cluster_b_cells = cluster_series[cluster_series == cluster_b]
        if cluster_b_cells.size > max_test_cell_population:
            cluster_b_cells = cluster_b_cells.sample(max_test_cell_population,
                                                     random_state=random_seed)
        cluster_b_cells = cluster_b_cells.index
        cells = cluster_a_cells | cluster_b_cells

        pair_adata = adata[cells, :].copy()
        pair_adata.obs['cluster'] = pair_adata.obs[cluster_col].astype(
            str).astype('category')

        sc.tl.rank_genes_groups(pair_adata,
                                groupby='cluster',
                                n_genes=top_n,
                                method='wilcoxon')
        gene_set = get_sig_features(pair_adata.uns['rank_genes_groups'])
        # drop na
        gene_set = set([i for i in gene_set if isinstance(i, str)])

        total_markers.update(gene_set)
        pair_marker_count[pair] = len(gene_set)
    return total_markers, pair_marker_count

In [11]:
total_markers = set()
pair_marker_counts = {}
with ProcessPoolExecutor(cpu) as executor:
    futures = []
    for chunk_start in range(0, len(cluster_pairs), chunk_size):
        pair_chunk = cluster_pairs[chunk_start:chunk_start + chunk_size]
        future = executor.submit(pairwise_tests, pair_chunk)
        futures.append(future)

    for future in as_completed(futures):
        genes, pair_marker_count = future.result()
        total_markers.update(genes)
        pair_marker_counts.update(pair_marker_count)

## Save results

In [12]:
total_markers = set([i for i in total_markers if isinstance(i, str)])
with open('Markers/rna.cluster_markers.txt', 'w') as f:
    f.write('\n'.join(total_markers))

In [13]:
pair_marker_counts = pd.Series(pair_marker_counts)
marker_counts = pair_marker_counts.reset_index()
marker_counts.columns = ['ClusterA', 'ClusterB', 'GeneCount']
marker_counts.to_csv('Markers/rna.cluster_pair_marker_counts.csv', index=None)

In [14]:
marker_counts[marker_counts['GeneCount'] < 3]

,ClusterA,ClusterB,GeneCount


In [17]:
adata = anndata.read_h5ad(adata_path)
marker_adata = adata[:, list(total_markers)]
marker_adata.write_h5ad('Markers/rna.cluster_markers.h5ad')
marker_adata

AnnData object with n_obs × n_vars = 7795 × 305 
    obs: 'Age', 'AnalysisPool', 'AnalysisProject', 'Bucket', 'CellConc', 'Cell_Conc', 'ChipID', 'Class', 'ClassProbability_Astrocyte', 'ClassProbability_Astrocyte,Immune', 'ClassProbability_Astrocyte,Neurons', 'ClassProbability_Astrocyte,Oligos', 'ClassProbability_Astrocyte,Vascular', 'ClassProbability_Bergmann-glia', 'ClassProbability_Blood', 'ClassProbability_Blood,Vascular', 'ClassProbability_Enteric-glia', 'ClassProbability_Enteric-glia,Cycling', 'ClassProbability_Ependymal', 'ClassProbability_Ex-Neurons', 'ClassProbability_Ex-Vascular', 'ClassProbability_Immune', 'ClassProbability_Immune,Neurons', 'ClassProbability_Immune,Oligos', 'ClassProbability_Neurons', 'ClassProbability_Neurons,Cycling', 'ClassProbability_Neurons,Oligos', 'ClassProbability_Neurons,Satellite-glia', 'ClassProbability_Neurons,Vascular', 'ClassProbability_OEC', 'ClassProbability_Oligos', 'ClassProbability_Oligos,Cycling', 'ClassProbability_Oligos,Vascular', 'Class

In [18]:
marker_counts.describe()

,GeneCount
count,66.000000
mean,39.272727
std,1.283570
min,33.000000
25%,39.000000
50%,40.000000
75%,40.000000
max,40.000000
